# ReAct

Ein _ReAct Agent_ ist ein KI-System, das "Reasoning and Acting" (Denken und Handeln) kombiniert. Dieses Verfahren basiert auf einem Paper: https://react-lm.github.io/.

- Abwechslung von Denken und Handeln: Der Agent wechselt zwischen Denkphasen, in denen er plant und schlussfolgert, und Handlungsphasen, in denen er Aktionen ausführt.
- Schrittweise Problemlösung: Komplexe Aufgaben werden in kleinere Schritte zerlegt und sequentiell bearbeitet.
- Anpassungsfähigkeit: Der Agent kann seinen Ansatz basierend auf Zwischenergebnissen oder neuen Informationen anpassen.
- Werkzeuge: ReAct Agenten können oft externe Tools oder Datenquellen in ihren Problemlösungsprozess einbinden.

Der Vorteil von diesem Verfahren ist, dass die Ergebnisse meist sehr viel besser sind, als wenn man einfach nur in einem One-Shot versucht die finale Antwort zu bekommen.

Jetzt werden wir noch tiefer in die automatische Trickkiste von LangChain graben, denn schlaue Leute haben sich schon Gedanken gemacht und wir machen uns das zunutze!
Wie zuvor bekommt der Agent Tools an die Hand. Wir nutzen einfach die Tools, die wir im letzten Abschnitt verfügbar gemacht haben.

Starten wir also mit der Initialisierung.

In [1]:
!pip install langchain langchain_openai python-dotenv duckduckgo-search wikipedia langchainhub


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")

# model = ChatOpenAI(model="gpt-4o")
model = AzureChatOpenAI(openai_api_version="2024-05-01-preview", azure_deployment="gpt-4o", temperature=0.5, streaming=True)

search = DuckDuckGoSearchResults()

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [search, wikipedia]

Jetzt werden wir den Agenten anlegen (lassen). Dabei nutzen wir alles an Automatismus, der geboten wird. So holen wir sogar den Prompt von einem _Hub_. Auf diesem Hub (https://smith.langchain.com/hub) gibt es eine Menge Usergenerierter-Prompts, die man bequem mit einem Befehl holen kann.

Zusätzlich nutzen wir eine Hilfsfunktion, um einen ReAct-Agenten zu erstellen.

In [10]:
# Wir holen uns den Prompt aus eine online Repository: https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Jetzt wo wir den Executor haben, können wir auch den Agenten schon mal mit der ersten Aufgabe beschäftigen.

In [11]:
agent_executor.invoke({"input": "Was ist LangChain?"})



> Entering new AgentExecutor chain...
LangChain könnte sich auf eine Technologie oder ein Produkt beziehen. Um eine genaue Antwort zu geben, sollte ich nach spezifischen Informationen suchen.

Action: duckduckgo_results_json
Action Input: "Was ist LangChain?"[snippet: LangChain macht es einfacher, Anwendungen auf Generative-AI-Basis zu entwickeln. Das sollten Sie über das Python- und JavaScript-Framework wissen., title: Was ist LangChain? - computerwoche.de, link: https://www.computerwoche.de/a/was-ist-langchain,3615211], [snippet: Build your own chatbot — use LangChain's LLM interfaces, prompt templates and memory options to build a chatbot with conversation history. Question-answering of specific documents — by ..., title: Introducing LangChain: a beginner's guide ️ - Medium, link: https://medium.com/@ellen.hoeven/introducing-langchain-a-beginners-guide-️-5e751622edb7], [snippet: In this blog, we cover LangChain: what it is, how it works, examples, & its ability to enhance applicat

{'input': 'Was ist LangChain?',
 'output': 'LangChain ist ein Framework für Python und JavaScript, das die Entwicklung von Anwendungen erleichtert, die auf generativen KI-Sprachmodellen basieren.'}

Das war eine relativ einfache Aufgabe. Aber man sieht schon, dass anstelle, dass der Agent einfach eine Suche lostritt, sich erstmal "Gedanken" macht, wie man zu der Lösung kommen könnte.

Lass uns den Agenten mit etwas komplizierteren Aufgabe beschäftigen.

In [12]:
agent_executor.invoke({"input": "Ich habe einige Holz-Paletten rumstehen und möchte daraus eine Gartenhütte bauen. Wie geht das?"})



> Entering new AgentExecutor chain...
Ich sollte nach einer Anleitung zur Erstellung einer Gartenhütte aus Holzpaletten suchen. Dafür kann ich entweder eine allgemeine Anleitung auf Wikipedia suchen oder spezifische Anleitungen und Baupläne über eine Websuche finden.

Action: duckduckgo_results_json
Action Input: Gartenhütte aus Holzpaletten bauen Anleitung[snippet: Schritt-für-Schritt-Anleitung Gartenhaus aus Paletten bauen. Wenn Sie ein Häuschen für den Garten bauen wollen und dabei auf Ziegelsteine, Bagger und anderes verzichten möchten, dann bietet sich ein Gartenhäuschen aus Europaletten an. Das ist nicht nur einfach, sondern auch voll trendy., title: Gartenhaus aus Paletten - so bauen Sie es selbst - HELPSTER, link: https://www.helpster.de/gartenhaus-aus-paletten-so-bauen-sie-es-selbst_232200], [snippet: Aus Paletten kannst du viele Möbel, DIY-Gartenmöbel oder Spielgeräte für Kinder bauen. Sowohl für den Innen- als auch für den Außenbereich gibt es zahlreiche Ideen, die du mit 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Es gibt einige hilfreiche Ressourcen, die Schritt-für-Schritt-Anleitungen zum Bau einer Gartenhütte aus Holzpaletten bieten. Hier sind einige nützliche Links:

1. [Gartenhaus aus Paletten - so bauen Sie es selbst - HELPSTER](https://www.helpster.de/gartenhaus-aus-paletten-so-bauen-sie-es-selbst_232200): Diese Seite bietet eine detaillierte Anleitung, wie man ein Gartenhaus aus Europaletten bauen kann. Es ist eine einfache und trendige Methode.

2. [Spielhaus selber bauen aus Paletten - Anleitung | OBI](https://www.obi.de/magazin/garten/freizeit/spielhaus-selber-bauen-paletten): Diese Anleitung zeigt, wie man ein Spielhaus aus Paletten bauen kann, was ähnliche Prinzipien wie beim Bau einer Gartenhütte beinhaltet.

3. [Gartenmöbel aus Paletten selber bauen - Tipps | OBI](h

In meinem Fall hat der Agent ganz schnell eine Anleitung gefunden und mir als Ergebnis präsentiert. Wo der Agent glänzt, ist wenn man eine Aufgabe vergibt, die nicht einfach durch eine einfache Suche direkt erledigt werden kann. Häufiger Usecase ist Recherchearbeit.



# Aufgabe

Jetzt haben wir uns einen ReAct-Agenten gebaut. Diese Agenten sind nützlich, wenn man z.B. Recherchen machen möchte.
Erstelle einen ReAct-Agenten mit einer RAG-Fähigkeit. D.h. du stellst ihm ein RAG-Tool zur Verfügung.

Lösungshinweise:

- Du musst, wie wir das schon gelernt haben, einen Retriever zunächst erstellen. Nutze gerne die beigelegten Dokumente als Wissensdatenbank.
- Aus dem Retriever musst du ein Tool bauen. Es gibt dafür eine Hilfsfunktion: _create\_retriever\_tool_ aus dem Paket _langchain.agents.agent\_toolkits_
- Denke daran, dass dieses Tool auch eine vernünftige Beschreibung braucht, damit der Agent weiß wozu das gut ist.



In [ ]:
# Hier kannst du deine Lösung programmieren